<a href="https://colab.research.google.com/github/someDeveloperDH/game/blob/main/RSP_CNN_dohyeon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 블로그

https://medium.com/@msmapark2/vgg16-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-very-deep-convolutional-networks-for-large-scale-image-recognition-6f748235242a

https://blog.naver.com/2hannseok/223229158970

https://diseny.tistory.com/entry/%ED%98%BC%EB%8F%99%ED%96%89%EB%A0%ACconfusion-matrix?category=906035


In [2]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
#필요한거 import
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import numpy as np
import os

In [4]:
base_dir = '/content/drive/MyDrive/RPS'  # 필요시 경로 수정
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# 이미지 크기 128x128로 설정 (100x100도 해봤는데 정확도 30퍼나옴..)
img_size = (128, 128)

# 데이터 증강
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
#generator 설정(_g)
train_g = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=16,  # 배치 크기 16으로 함
    class_mode='categorical'
)

val_g = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=16,
    class_mode='categorical'
)

Found 1950 images belonging to 3 classes.
Found 270 images belonging to 3 classes.


In [6]:
# 데이터셋에 맞게 스텝 조정
epochstep = train_g.samples // train_g.batch_size
val_step = val_g.samples // val_g.batch_size

# VGG16 모델 사용
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

#모든 레이어 freeze
for layer in vgg_base.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
model = Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=1e-5),  # 학습률을 줄임
    metrics=['accuracy']
)

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/RPS/model_checkpoint.keras', save_best_only=True, monitor='val_loss')

history = model.fit(
    train_g,
    steps_per_epoch=epochstep,
    epochs=10,
    validation_data=val_g,
    validation_steps=val_step,
    callbacks=[early_stopping, checkpoint]
)


# 혼동 행렬 계산
Y_pred = model.predict(val_g, val_g.samples // val_g.batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)

cm = confusion_matrix(val_g.classes, y_pred)
print(cm)


Epoch 1/10
121/121 [==============================] - 37s 299ms/step - loss: 1.1587 - accuracy: 0.4105 - val_loss: 0.8617 - val_accuracy: 0.6875
Epoch 2/10
121/121 [==============================] - 22s 176ms/step - loss: 0.9142 - accuracy: 0.5760 - val_loss: 0.7235 - val_accuracy: 0.7891
Epoch 3/10
121/121 [==============================] - 22s 181ms/step - loss: 0.7848 - accuracy: 0.6587 - val_loss: 0.6299 - val_accuracy: 0.8281
Epoch 4/10
121/121 [==============================] - 21s 172ms/step - loss: 0.7163 - accuracy: 0.7208 - val_loss: 0.5655 - val_accuracy: 0.8555
Epoch 5/10
121/121 [==============================] - 22s 180ms/step - loss: 0.6505 - accuracy: 0.7554 - val_loss: 0.5261 - val_accuracy: 0.8555
Epoch 6/10
121/121 [==============================] - 23s 187ms/step - loss: 0.5901 - accuracy: 0.7813 - val_loss: 0.4647 - val_accuracy: 0.8906
Epoch 7/10
121/121 [==============================] - 21s 176ms/step - loss: 0.5532 - accuracy: 0.8020 - val_loss: 0.4305 - val_ac